# Big Data Project 20211

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 146 kB 965 kB/s            
     |████████████████████████████████| 15.7 MB 4.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 11.5 MB 75.8 MB/s            
     |████████████████████████████████| 26.4 MB 78.4 MB/s            
     |████████████████████████████████| 306 kB 91.9 MB/s            
     |████████████████████████████████| 39.8 MB 130 kB/s             
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1315 sha256=cf53589983cd71927994d3ec7117c6863973f50a9bc81f5ff2c3b674a53c2c71
  Stored in directory: /root/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.driver.memory","16G").\
        config("spark.driver.maxResultSize", "0").\
        config("spark.kryoserializer.buffer.max", "2000M").\
        config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.1").\
        getOrCreate()
spark

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a9b3e03b-1d16-40cb-b5f6-287a2f5120d9;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.0.1 in central
	found com.typesafe#config;1.3.0 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3

In [2]:
spark.version

'3.0.0'

In [2]:
# dataset = spark.read \
#       .option("header", True) \
#       .json("data/Prime_Pantry.json")
dataset = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option('quote', '"') \
      .option('escape', '"') \
      .csv("hdfs://namenode:9000/data/input/reviews_0.csv")
      

In [4]:
# filter rows with include positive and negative reviews
dataset = dataset.filter(dataset.overall.isin([1, 2, 3, 4, 5]))
# count positive and negative reviews
dataset.groupBy('overall').count().show()

+-------+------+
|overall| count|
+-------+------+
|      3| 97031|
|      5|465478|
|      1|107080|
|      4|149331|
|      2| 64718|
+-------+------+



In [5]:
from pyspark.sql import functions as F
# apply overall > 3.0 to positive reviews, else negative
dataset = dataset.withColumn('target', F.when(F.col('overall') > 3.0, 'positive').otherwise('negative'))
dataset.show(5)
# count positive and negative reviews
dataset.groupBy('target').count().show()

+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-----+--------+
|overall|verified| reviewTime|    reviewerID|      asin|reviewerName|          reviewText|             summary|unixReviewTime|vote|style|image|  target|
+-------+--------+-----------+--------------+----------+------------+--------------------+--------------------+--------------+----+-----+-----+--------+
|      5|    True|10 20, 2014|A1D4G1SNUZWQOT|7106116521|       Tracy|Exactly what I ne...|perfect replaceme...|    1413763200|null| null| null|positive|
|      2|    True|09 28, 2014|A3DDWDH9PX2YX2|7106116521|   Sonja Lau|I agree with the ...|I agree with the ...|    1411862400| 3.0| null| null|negative|
|      4|   False|08 25, 2014|A2MWC41EW7XL15|7106116521|    Kathleen|Love these... I a...| My New 'Friends' !!|    1408924800|null| null| null|positive|
|      2|    True|08 24, 2014|A2UH2QQ275NV45|7106116521| Jodi Stoner| too tiny an 

+--------+------+
|  target| count|
+--------+------+
|positive|614809|
|negative|268829|
+--------+------+



In [24]:
dataset = dataset[['overall', 'reviewText']]
dataset.show(5)

+--------+--------------------+
| overall|          reviewText|
+--------+--------------------+
|positive|Im a die hard Dad...|
|negative|This movie was a ...|
|negative|I just sat in the...|
|positive|Just saw this fil...|
|negative|I consider myself...|
+--------+--------------------+
only showing top 5 rows



In [4]:
from pyspark.sql.functions import rand 
dataset = dataset.orderBy(rand()) # Shuffle

In [29]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

In [19]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], seed = 100)
trainingData.show(10)
testData.show(10)

+--------------------+--------+--------------------+--------------------+--------------------+-----+
|          reviewText| overall|               words|            filtered|            features|label|
+--------------------+--------+--------------------+--------------------+--------------------+-----+
|"Dead Man Walking...|positive|[dead, man, walki...|[dead, man, walki...|(10000,[0,1,2,3,4...|  1.0|
|"Fat Girls" is am...|negative|[fat, girls, is, ...|[fat, girls, is, ...|(10000,[0,1,2,3,4...|  0.0|
|"Riders of Destin...|negative|[riders, of, dest...|[riders, of, dest...|(10000,[0,1,2,3,5...|  0.0|
|"Valley Girl" lau...|positive|[valley, girl, la...|[valley, girl, la...|(10000,[0,1,2,3,4...|  1.0|
|*Spoilers herein*...|negative|[spoilers, herein...|[spoilers, herein...|(10000,[0,1,2,3,4...|  0.0|
|...I saw this on ...|positive|[i, saw, this, on...|[i, saw, this, on...|(10000,[0,1,2,3,4...|  1.0|
|<br /><br />In th...|negative|[br, br, in, the,...|[br, br, in, proc...|(10000,[0,1,2,3,5.

In [30]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

In [33]:

predictions = cvModel.transform(testData)
predictions = predictions.select("reviewText","overall","probability","label","prediction")
predictions.show(n = 20, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                    reviewText| overall|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|"Throw Momma From the Train...|positive|[0.004367934986061326,0.995...|  1.0|       1.0|
|A Formula For Murder isn't ...|positive|[0.03301721350734545,0.9669...|  1.0|       1.0|
|A less than redemptive hunk...|negative|[0.7608247074504314,0.23917...|  0.0|       0.0|
|After the return of "horror...|positive|[0.4902780675040531,0.50972...|  1.0|       1.0|
|Although others have commen...|positive|[0.40823599145099165,0.5917...|  1.0|       1.0|
|As I saw the movie I was re...|positive|[0.47973990133809774,0.5202...|  1.0|       1.0|
|Before the Internet this mo...|negative|[0.8852763163972478,0.11472...|  0.0|       0.0|
|Best animated movie ever ma...|positive|[0.13445387145687182,0.8655...|  1.0|       1.0|
|Bizarre h

In [32]:
evaluator.evaluate(predictions)

0.8644098421108525

In [ ]:
predictions = predictions.toPandas()

## Evaluate Model

In [36]:

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report
print (classification_report(predictions['prediction'], predictions['label']))

              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90      4931
         1.0       0.91      0.89      0.90      5101

    accuracy                           0.90     10032
   macro avg       0.90      0.90      0.90     10032
weighted avg       0.90      0.90      0.90     10032

